Reference: Piasecka, E. et al. Natural variation in the parameters of innate immune cells is preferentially driven by genetic factors resource. Nat. Immunol. 19, 302-314 (2018). Piasecka, B. et al. Distinctive roles of age, sex, and genetics in shaping transcriptional variation of human immune responses to microbial challenges. Proc. Natl. Acad. Sci. 115, E488-E497 (2018). и http://www.milieuinterieur.fr/

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
import numpy as np
import itertools
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import scale, OneHotEncoder
from sklearn.metrics import explained_variance_score, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
%matplotlib inline

Данный датасет представляет собой рнк-секвенирование иммуного ответа пациентов на различные стимулы  (Escherichia coli, BCG, Staphylococcus aureus, SEB, Candida albicans and Influenza virus). В данном исследование были секвенированы 560 генов. Основная задача - оценить, насколько может быть вариабелен иммунный ответ в рамках популяции (пусть даже выборка тут небольшая)

Первые колонки, которые в своем названии содержат MFI - это не гены, это усредненный показатель наличиние антитела Н в популяции клеток К. Например, CD38_MFI_in_Bcells  показывает уровень CD38 в B-лимфоцитах. Данные показатели и будут показывать "уровень" имунного ответа. Чем больше этот показатель - тем больше клеток такого типа мы наблюдаем. 
В данной работе мы сосредоточимся на следующийх генах - IFNA2 (возраст) IFNG(возраст),  MAPK14, GATA3

In [ ]:
dt = pd.read_csv('dataset_merged.txt',sep='\t')
dt.head()

Проведем небольшую разминку - построим графики, посмотрим на распределения, проведем тесты, посчитаем статистики. 

У нас есть дополнительная информация. Самые интересные из колонок:
 - Age
 - PhysicalActivity
 - Sex
 - MetabolicScore
 - HoursOfSleep
 - UsesCannabis
 - Smoking
 - BMI

 Посмотрим на распределение возрастов и пола

In [ ]:
metadata_columns = ['id','Age','PhysicalActivity','Sex','MetabolicScore','BMI','HoursOfSleep','UsesCannabis',\
                    'Smoking','Employed','Education','DustExposure']
ids_description  = dt.drop_duplicates('id').drop(['stimulus'],axis=1)[metadata_columns]
ids_description.head()

In [ ]:
sns.pairplot(ids_description[['Age','PhysicalActivity','MetabolicScore','BMI','HoursOfSleep']]);
plt.show()
sns.catplot(x="Sex", y="Age", hue="Smoking",data=ids_description,kind="violin");

В принципе, данные довольно сбалансированные. Не удивительно, ведь авторы исследования так и пытались сделать. 
Теперь начнем смотреть на экспрессии генов и их связь с возрастом\полом\статусом.

3.1 Сравнение экспресии IFNG у мужчин и женщин

3.1.1 Рассмотрим распределение экспресси IFNG у женщин (со стимулом - S.aureus). На что похоже это распределение? Как это проверить? 

In [ ]:
mask = (dt['Sex']=='Female') & (dt['stimulus']=='S.aureus')
sns.distplot(dt.loc[mask,'IFNG']);

Какое удивительное нормальные распределение! Давайте проверим, что оно нормальное

In [ ]:
k2, p = sc.stats.shapiro(dt.loc[mask,'IFNG'].values)
print('p value {}'.format(p))
sc.stats.probplot(dt.loc[mask,'IFNG'].values,plot=plt);

 Оценим параметры этого распределения.

In [ ]:
d_var = np.var(dt.loc[mask,'IFNG'].values)
d_mean = np.mean(dt.loc[mask,'IFNG'].values)
print('mean {}, var {}'.format(d_mean, d_var))

Задача 1. Сделать аналогичные шаги для мужчин (со стимулом - S.aureus)

In [ ]:
#Task 1 

Задача 2. Давайте используем бустреп чтобы оценить доверительный интервал для среднего уровня экспресии гена
GATA3 у мужчин и женщин. Используем 1000 итераций, будем семплить по 100 точек. Давайте условимся, что в этот раз работаем со стимулом E.coli

In [ ]:
#Task 2 solution

Выглядит так, будто экспрессия и правда различна. Давайте посмотрим на совместное распределение. В этом нам поможет violin plot

In [ ]:
#plot IFNG by sex

Сложно сказать однозначно... Что тут можно сделать? 

Задача 3. Проверим, является ли различие в средней экспресии IFNG у мужчин и женщин при E.coli статистически значимым.
Каким тестом тут лучше воспользоваться и почему?  (при уровне значимости $\alpha  = 0.05$)

In [ ]:
#Task 3 solution

Задача 4. А интересно, у скольки еще генов различие в средней экспресии у мужчин и женщин при E.coli статистически значимым (при уровне значимости $\alpha  = 0.05$). Не будем усложнять себе жизнь и воспользуемся поправкой Бонферрони

In [ ]:
#Task 4 solution

Отлично, но вернемся к IFNG. Добавим к рассмотрниею следующие параметры - Age, PhysicalActivity, BMI, Smoking, Education, LivesWithPartner. Как можно заметить, некоторые из них категориальные, некоторые - числовые. Тем интересней!

Задача 5. Рассмотрите уровень экспресии IFNG (при E.coli)  в зависимости от каждого из вышеназванных параметров. 
Какой вывод можно сделать? А если дополнительно разбить с учетом пола? 

In [ ]:
#Task 5 solution

Задача 6. Для того, чтобы рассматривать экспрессию между несколькими группами, нам понадобится знание диспресионного анализа. 
Проверьте, можем ли мы утверждать, что уровень экспресии IFNG не зависит от курения (без разбиения по полу) (используейте пакет $statsmodels$). Дополнительно проведите Tukey hsd чтобы оценить разницу между группами

In [ ]:
#Task 6 solution

В последующих шагах по постараемся воспроизвести основные результаты статьи  Piasecka, E. et al. Natural variation in the parameters of innate immune cells is preferentially driven by genetic factors resource. Nat. Immunol. 19, 302-314 (2018), опубликованной в Nature Genetics. Далее мы работаем с стимулом NS, то есть с изначальным иммуным ответом.

In [ ]:
dt_ns  = dt.loc[dt['stimulus']=='NS',:]

Большое Задание 1. Рассмотрите зависимости между предложенными негентическими переменными в этом датасете. Для это сделаете попарные линейные модели и в качестве зависимости возьмите $R^2$. При работе с категориальными переменными, используйте OHE стратегию и используйте их только как предикторы. (Хотя в случае бинарного лейбла можно сделать логрегрессию)

In [ ]:
nongenetic_columns = ['Age', 'OwnsHouse',
                      'PhysicalActivity', 'Sex', 'LivesWithPartner', 'LivesWithKids',
                      'BornInCity', 'Inbreeding', 'BMI', 'CMVPositiveSerology', 'FluIgG',
                      'MetabolicScore', 'LowAppetite', 'TroubleConcentrating',
                      'TroubleSleeping', 'HoursOfSleep', 'Listless', 'UsesCannabis',
                      'RecentPersonalCrisis', 'Smoking', 'Employed', 'Education',
                      'DustExposure', 'Income',
                      'DepressionScore', 'HeartRate',
                      'Temperature']

In [ ]:
#Big task 1 solution

Большое задание 1.1. Теперь мы посмотрим на корреляцию между различными MFI. Это нужно, чтобы дальше мы взяли в работу только нескореллированные признаки

In [ ]:
MFI_cols = [x for x in dt_ns.columns.values if 'MFI_' in x]

In [ ]:
#Big task 1.1 solution 

Большое задание 2. Теперь сделаем регресионные модели для каждого из этих показателей.

In [ ]:
factors = ['Age', 'Sex', 'BMI', 'CMVPositiveSerology', 'MetabolicScore', 'Smoking']
dt_ns_regres = dt_ns[factors+noncorrelated_features]

In [ ]:
#Big task 2 solutions